# Recherche d'articles similaires

In [25]:
import pandas as pd
from sentence_transformers import SentenceTransformer


import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup

import html 
import re
import numpy as np

In [3]:
## Import Data
df_mots = pd.read_csv("./data/b_mot_cle.csv", encoding="utf8")
df_mots.info()
df_mot_entite = pd.read_csv("./data/num_mot_entite.csv", encoding="utf8")
df_mot_entite.info()
df_publis = pd.read_csv("./data/num_publis.csv", encoding="utf8")
df_publis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2542 non-null   int64 
 1   mot     2542 non-null   object
 2   valid   2542 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 59.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3093 entries, 0 to 3092
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           3093 non-null   int64  
 1   id_mot       3093 non-null   int64  
 2   id_publi     2234 non-null   float64
 3   id_ecriture  0 non-null      float64
 4   id_user      748 non-null    float64
 5   id_revue     76 non-null     float64
 6   id_numero    35 non-null     float64
 7   valid        3093 non-null   int64  
dtypes: float64(5), int64(3)
memory usage: 193.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data column

In [24]:
df_publis.head(50)

,id,id_revue,id_numero,type,titre,soustitre,position,resume,article,id_ecriture_parent,slug,url,valid
0,239,1,35,0,Note de recherche-Le rôle de l’entrepreneuriat...,Sous titre par défaut,6,&lt;p style=&quot;text-align:justify; margin-b...,&lt;h2&gt;Introduction&lt;/h2&gt;\r\n\r\n&lt;p...,147,note-de-recherche-le-role-de-l-entrepreneuriat...,NaN,1
1,240,1,35,0,"Transformations organisationnelles, développem...",un cas d'adaptation au changement climatique,1,&lt;p&gt;&lt;strong&gt;R&amp;eacute;sum&amp;ea...,&lt;h2&gt;INTRODUCTION&lt;/h2&gt;\r\n\r\n&lt;p...,170,transformations-organisationnelles-developpeme...,NaN,1
2,241,1,35,0,Comment la digitalisation du don participatif ...,Sous titre par défaut,3,&lt;p&gt;La recomposition des &amp;eacute;cosy...,&lt;h1 style=&quot;text-align: center;&quot;&g...,182,comment-la-digitalisation-du-don-participatif-...,NaN,1
3,242,1,35,0,La résilience urbaine inclusive,Les stratégies de résilience urbaine en direct...,2,&lt;p&gt;La tendance mondiale de l&amp;#39;urb...,&lt;h2&gt;Introduction&lt;/h2&gt;\r\n\r\n&lt;p...,243,la-resilience-urbaine-inclusive,NaN,1
4,243,1,35,0,La territorialisation comme condition de durab...,Sous titre par défaut,5,&lt;p&gt;R&amp;eacute;sum&amp;eacute;&lt;/p&gt...,&lt;h2 style=&quot;margin-left: 1pt; margin-ri...,251,la-territorialisation-comme-condition-de-durab...,NaN,1
5,244,1,35,0,Evaluer les capacités transformationnelles du ...,Sous titre par défaut,4,&lt;p style=&quot;text-align:justify; margin-b...,&lt;p&gt;&lt;span style=&quot;font-size:12.0pt...,261,evaluer-les-capacites-transformationnelles-du-...,NaN,1
6,248,1,35,1,Durabilité et transformation des organisations...,NaN,0,&lt;p&gt;-&lt;/p&gt;\r\n,&lt;p&gt;Le d&amp;eacute;veloppement durable e...,0,durabilite-et-transformation-des-organisations...,NaN,0
7,1598,1,64,0,La communication interne et le succès des syst...,Sous titre par défaut,0,&lt;p&gt;&lt;span style=&quot;font-size:11pt&q...,&lt;p&gt;Article&lt;/p&gt;\r\n\r\n&lt;h2 style...,1126,la-communication-interne-et-le-succes-des-syst...,NaN,1
8,1599,1,64,0,LA PLACE DE LA COMMUNICATION INTERNE DANS LE M...,Sous titre par défaut,1,&lt;p&gt;R&amp;eacute;sum&amp;eacute;&lt;/p&gt...,&lt;p&gt;LA PLACE DE LA COMMUNICATION INTERNE ...,1160,la-place-de-la-communication-interne-dans-le-m...,NaN,1
9,1600,1,64,0,Les effets positifs de l'usage des Technologie...,TIC dans la GRH : Quel impact sur la performan...,2,&lt;h3&gt;R&amp;eacute;sum&amp;eacute;&lt;/h3&...,&lt;h3&gt;Introduction&lt;/h3&gt;\r\n\r\n&lt;p...,1234,les-effets-positifs-de-l-usage-des-technologie...,NaN,1


In [6]:

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return html.unescape(cleantext)


data = list(df_publis.titre.dropna())
data = [cleanhtml(BeautifulSoup(string).text) for string in data]
print(len(data))
data[0]

1815


'Note de recherche-Le rôle de l’entrepreneuriat coopératif dans l’inclusion socioéconomique des femmes : Cas de la coopérative féminine des produits de la mer de Douira « COFEPROMER »'

In [30]:
df = df_publis.loc[:,["titre","resume"]]

df["titre"] = df.titre.apply(lambda x: cleanhtml(BeautifulSoup(x).text))
df["resume"] = df.resume.apply(lambda x: cleanhtml(BeautifulSoup(x).text) if x is not np.NaN else x)
df.head()

,titre,resume
0,Note de recherche-Le rôle de l’entrepreneuriat...,"Au niveau de la région Souss Massa, la collect..."
1,"Transformations organisationnelles, développem...",Résumé : L'enjeu du développement durable peut...
2,Comment la digitalisation du don participatif ...,La recomposition des écosystèmes locaux de la ...
3,La résilience urbaine inclusive,La tendance mondiale de l'urbanisation mais au...
4,La territorialisation comme condition de durab...,Résumé\r\n\r\nL’inscription dans un territoire...


In [60]:


df_publis["titre"] = df_publis.titre.apply(lambda x: cleanhtml(BeautifulSoup(x).text))
df_publis["resume"] = df_publis.resume.apply(lambda x: cleanhtml(BeautifulSoup(x).text) if x is not np.NaN else x)

df = df_publis.loc[:,["titre","resume"]].dropna()

text = df.titre + df.resume
print(len(text))
print(len(text.dropna()))


In [39]:
"""
This example demonstrates how we can perform semantic search for scientific publications.
As model, we use SPECTER (https://github.com/allenai/specter), which encodes paper titles and abstracts 
into a vector space.
When can then use util.semantic_search() to find the most similar papers.
Colab example: https://colab.research.google.com/drive/12hfBveGHRsxhPIUMmJYrll2lFU4fOX06
"""


from sentence_transformers import SentenceTransformer, util


paper_texts = list(text)

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')



#Compute embeddings for all papers
corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True)






Paper: Specializing Word Embeddings (for Parsing) by Information Bottleneck
Most similar papers:


NameError: name 'papers' is not defined

In [85]:
#We define a function, given title & abstract, searches our corpus for relevant (similar) papers

def search_papers(title, abstract):
    query_embedding = model.encode(title+' '+abstract, convert_to_tensor=True)

    search_hits = util.semantic_search(query_embedding, corpus_embeddings)
    search_hits = search_hits[0]  #Get the hits for the first query

    print("\n\nPaper:", title)
    print("Most similar papers:")
    for hit in search_hits:
        related_paper = df.iloc[hit['corpus_id'],:]
        
        print("{:.2f}\t\t{}".format(hit['score'], related_paper['titre']))
        print("ID", df_publis[df_publis.titre == related_paper['titre']].id.values[0])



In [100]:
df = df.dropna()
paper = df.iloc[1]
print("Papier recherché: ", paper.titre)
# search_papers(paper.titre, paper.resume)
# df[df.id==240]

Papier recherché:  Transformations organisationnelles, développement durable, et multiplicité des temporalités d'action


In [101]:
%time
search_papers(paper.titre, paper.resume)

Wall time: 0 ns


Paper: Transformations organisationnelles, développement durable, et multiplicité des temporalités d'action
Most similar papers:
1.00		Transformations organisationnelles, développement durable, et multiplicité des temporalités d'action
ID 240
0.93		Transformations organisationnelles, développement durable, et multiplicité des temporalités d'action. Un cas d'adaptation au changement climatique
ID 1875
0.87		La position des sociologues dans le processus de développement :Approche systémique et approche participative.
ID 765
0.85		Analyse critique de la gestion du changement organisationnel dans une organisation en modernisation. 
ID 179
0.85		Construction, circulation et praxis de l'anthropocène
ID 128
0.84		Approche théorique du développement d'organisations territoriales intelligentes appréhendant la complexité selon une approche intégrative de données
ID 1833
0.84		Gestion des données de recherche et thick data : le cas d’une recherche-action sur la médiation numériq

"Transformations organisationnelles, développement durable, et multiplicité des temporalités d'action"

In [73]:
df_publis[df_publis.id == 240]

,id,id_revue,id_numero,type,titre,soustitre,position,resume,article,id_ecriture_parent,slug,url,valid
1,240,1,35,0,"Transformations organisationnelles, développem...",un cas d'adaptation au changement climatique,1,Résumé : L'enjeu du développement durable peut...,&lt;h2&gt;INTRODUCTION&lt;/h2&gt;\r\n\r\n&lt;p...,170,transformations-organisationnelles-developpeme...,NaN,1


In [44]:
## Création du pipeline

1. Sauvegarde de l'embedding du corpus
2. Récupération de l'id de l'article retrouvé

In [79]:
import torch
torch.save(corpus_embeddings, 'corpus_embeddings.pt') 
torch.load('corpus_embeddings.pt')

tensor([[-1.3187,  0.4186,  0.0932,  ...,  0.0347,  0.1581,  1.2911],
        [-0.6885, -0.2902,  0.4928,  ..., -0.4240,  0.5615,  1.3186],
        [-0.4582, -0.0873,  0.0364,  ..., -0.1120, -0.1077,  0.7109],
        ...,
        [-0.0942,  1.0137,  0.0746,  ..., -0.1828,  0.2845,  0.4074],
        [-0.1906,  0.3226, -0.4037,  ...,  0.0521, -0.0833,  1.2705],
        [ 0.3996,  0.6050,  0.1530,  ..., -0.0434,  0.3501,  0.5134]])

In [80]:
corpus_embeddings.size()

torch.Size([771, 768])